In [1]:
import numpy as np
import pandas as pd

In [89]:
# Lendo os dados necessários
df_heart = pd.read_csv('heart.dat', header=None, delim_whitespace = True)
df_spam = pd.read_csv('spambase.data', header=None)

In [212]:
#Definindo função para calcular a PDF
def pdf_mv(x, mu, cov):
    '''
    Calcular a PDF multivariada
    Argumentos:
    x = vetor numpy (d x 1)
    mu = vetor de médias (d x 1)
    cov = matriz de covariancia (d x d)
    Retorna: Probabilidade de está contida na pdf com os argumentos passados.
    '''
    assert(mu.shape[0] > mu.shape[1]), 'mu deve ser um vetor fileira'
    assert(x.shape[0] > x.shape[1]), 'x deverá ser um vetor fileira'
    assert(cov.shape[0] == cov.shape[1]), 'matriz de covariancia deve ser quadrada'
    assert(mu.shape[0] == cov.shape[0]), 'erro de dimensão'
    assert(x.shape[0] == cov.shape[0]), 'erro de dimenssão'
    parte1 = 1/(((2* np.pi)**(len(mu)/2))*(np.linalg.det(cov)**(1/2)))
    parte2 = (-1/2) * ((x-mu).T.dot(np.linalg.inv(cov))).dot((x-mu))
    return float(parte1 * np.exp(parte2))

In [213]:
# Definindo função que aplica o teorema de Bayes [3]
def classe(x, c1, c2, px1, px2, cor1=0, cor2=0):
    '''
    Aplicar teorema de bayes
    Argumentos:
    x = vetor numpy a ser classificado (d x 1)
    c1 = vetor de médias do conjunto de dados 1 (d x 1)
    c2 = vetor de médias do conjunto de dados 2 (d x 1)
    px1 = probabilidade de x ser da classe 1
    px2 = probabilidade de x ser da classe 2
    cor1 = matriz de covariancia 1  (d x d)
    cor2 = matriz de covariancia 2  (d x d)
    Retorna: 0 ou 1 dependendo de qual PDF (classe) pretencer
    '''
    p1 = pdf_mv(x, c1, cor1)
    p2 = pdf_mv(x, c2, cor2)
    return float(px1*p1)>float(px2*p2)

In [214]:
# Função para fazer divisão de dados de treino e teste [1]
def train_test(df, train_v = 0.9):
    '''
    Fazer divisão em dados de teste e treino
    Argumentos:
    df = dataframe que se deseja dividir
    train_v = Porcentagem de dados de treino (deve ser menor que 1)
    Retorna: Vetores X, Y de treino e teste adequadamente distribuidos 
    '''
    assert(train_v<=1), 'train_v deve ser menor que 1'
    dados = df.to_numpy()
    np.random.shuffle(dados)
    train, test = dados[:int(train_v*len(dados))], dados[:-int(train_v*len(dados))]
    X_train, y_train = train[:, :train.shape[1]-1], train[:,train.shape[1]-1:]
    X_test, y_test = test[:, :train.shape[1]-1], test[:,train.shape[1]-1:]
    return (X_train, y_train, X_test, y_test)

In [215]:
# Função para calcular a média, desvio padrão, correlação e probabilidade merginal [2]
def media_dp_cor_pm(X_train, y_train):
    '''
    Calcular a média, desvio padrão, correlação e probabilidade merginal
    Argumentos:
    X_train = Matriz com os parametros X[n] da função analizada
    Y_train = Classificação dos dados
    Retorna: Média das duas classes, desvio padrão das duas classes, correlação das duas classes, probabilidade marginal das duas classes.
    '''
    xc1, xc2 = [], []
    for x, y in zip(X_train, y_train):
        if y == 1:
            xc1.append(x)
        else:
            xc2.append(x)
    xc1, xc2 = np.array(xc1), np.array(xc2)
    mean1 = np.mean(xc1, axis = 0)
    mean2 = np.mean(xc2, axis = 0)
    mean1 = mean1.reshape(mean1.shape[0], 1)
    mean2 = mean2.reshape(mean2.shape[0], 1)

    sd1 = np.std(xc1, axis = 0)
    sd2 = np.std(xc2, axis = 0)

    cor1 = (np.cov(np.transpose(xc1)))
    cor2 = (np.cov(np.transpose(xc2)))

    pxc1 = len(xc1)/(len(xc1)+len(xc2))
    pxc2 = len(xc2)/(len(xc1)+len(xc2))
    return (mean1, mean2, sd1, sd2, cor1, cor2, pxc1, pxc2)

In [216]:
X_train, y_train, X_test, y_test = train_test(df_spam, 0.9)
mean1, mean2, sd1, sd2, cor1, cor2, pxc1, pxc2 = media_dp_cor_pm(X_train, y_train)
print(f'Probabilidade marginal de ser da classe 1: {pxc1}\nProbabilidade marginal de ser da classe 2: {pxc2}')

Probabilidade marginal de ser da classe 1: 0.39565217391304347
Probabilidade marginal de ser da classe 2: 0.6043478260869565


In [217]:
# Definindo em qual classe estão os pontos de teste
classificador=[]
for test in X_test:
    test = test.reshape(test.shape[0],1)
    classificador.append(classe(x=test, c1=mean1, c2=mean2, px1=pxc1, px2=pxc2, cor1=cor1, cor2=cor2))

In [218]:
certos = 0
errados = 0

for classe_prevista, classe_real in zip(classificador, y_test):
    if (classe_prevista == classe_real) or (classe_prevista == 0 and classe_real == 2):
        certos += 1
    else:
        errados += 1
print(f'Numero de acertos: {certos}\nNúmeros de erros: {errados}\nPorcentagem de acertos: {certos/(certos+errados)*100:.2f}%')

Numero de acertos: 377
Números de erros: 84
Porcentagem de acertos: 81.78%


In [219]:
classificador=[]
for train in X_train:
    train = train.reshape(test.shape[0],1)
    classificador.append(classe(x=train, c1=mean1, c2=mean2, px1=pxc1, px2=pxc2, cor1=cor1, cor2=cor2))

In [220]:
certos = 0
errados = 0

for classe_prevista, classe_real in zip(classificador, y_train):
    if (classe_prevista == classe_real) or (classe_prevista == 0 and classe_real == 2):
        certos += 1
    else:
        errados += 1
print(f'Numero de acertos: {certos}\nNúmeros de erros: {errados}\nPorcentagem de acertos: {certos/(certos+errados)*100:.2f}%')

Numero de acertos: 3455
Números de erros: 685
Porcentagem de acertos: 83.45%
